<a href="https://colab.research.google.com/github/SimeonHristov99/ML_21-22/blob/main/unsupervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsupervised Learning

In **unsupervised learning**, the dataset is a collection of **unlabeled examples** $\{\textbf{x}_i\}_{i=0}^N$
Again, **x** is a feature vector, and the goal of an unsupervised learning algorithm is
to create a model that takes a feature vector **x** as input and either transforms it into
another vector or into a value that can be used to solve a practical problem.

## Types of problems

- **clustering**: the model returns the id of the cluster for each feature vector in the dataset;
- **dimensionality reduction**: the output of the model is a feature vector that has fewer features than the input **x**;
- **outlier detection**: the output is a real number that indicates
how **x** is different from a “typical” example in the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

from sklearn.metrics.cluster import adjusted_rand_score

In [ ]:
FIGSIZE = (12, 10)

plt.rc('figure', figsize=FIGSIZE)
sns.set_style('whitegrid')

## Clustering

**Clustering** is a problem of learning to assign a label to examples by leveraging an unlabeled
dataset. Because the dataset is completely unlabeled, deciding on whether the learned model
is optimal is much more complicated than in supervised learning.

> **Definition**: A cluster is a group of similar points.

### K-Means

- Divides the dataset into `k` similar groups.
- How does it work? First, you choose `k` — the number of
clusters. Then you randomly put `k` feature vectors, called centroids, to the feature space.
We then compute the distance from each example `x` to each centroid `c` using some metric,
like the Euclidean distance. Then we assign the closest centroid to each example (like if we labeled each example with a centroid id as the label). For each centroid, we calculate the average feature vector of the examples labeled with it. These average feature vectors become the new locations of the centroids. We recompute the distance from each example to each centroid, modify the assignment and
repeat the procedure until the assignments don’t change after the centroid locations were
recomputed. The model is the list of assignments of centroids IDs to the examples.
- Advantages:
  - fast;
  - scales well to large number of samples;
  - has been used across a large range of application areas in many different fields.
- Disadvantages:
  - the initial position of centroids influence the final positions, so two runs of k-means can
result in two different models;
  - the value of `k`, the number of clusters, is a hyperparameter that can be tuned. There are no proven optimal techniques for selecting `k`. Most of them require an `educated guess` by looking at some
metrics or by examining cluster assignments visually;
  - cannot deal with non-linear cluster shapes.

In [ ]:
X, y = make_blobs(random_state=1)
X

In [ ]:
y

In [ ]:
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1])

We'll try to get the following diagram but by using K-Means instead of the true labels `y`.

In [ ]:
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)

> **Note**: It may be obvious but the `.fit` method **does not** accept `y`.

#### Trying out an example

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=2)
kmeans.fit(X)
y_pred = kmeans.predict(X)
y_pred

> **Note**: `y_pred` does not necessarily equal `y`. 

In [ ]:
y

In [ ]:
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y_pred)

#### Measuring performance

**If we had the true labels**, we could measure the performance of our model by using the adjusted Rand index. It is a function that measures the similarity of the two assignments, ignoring permutations. Note that this is not accuracy - it is a measure of similarity.

More on Rand index: https://www.youtube.com/watch?v=6rjTIwn0yWI, https://scikit-learn.org/stable/modules/clustering.html#adjusted-rand-score

**If we didn't know the true labels**, we could use the built-in `.score` method. It returns the negative of the K-means objective. 

**K-Means Objective**

The objective in the K-means is to reduce the sum of squares of the distances of points from their respective cluster centroids. It has other names like J-Squared error function, J-score or within-cluster sum of squares. This value tells how internally coherent the clusters are (the less the better).

> **Note**: Although the result from `.score` might be low, this **does not guarantee** that the clusters are appropriate.

In [ ]:
adjusted_rand_score(y, y_pred)

In [ ]:
kmeans.score(X)

In [ ]:
kmeans.cluster_centers_

If we don't know the number of clusters, K-Means is not that helpful.

In [ ]:
kmeans = KMeans(n_clusters=2)
y_pred = kmeans.fit_predict(X)
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y_pred)

print(adjusted_rand_score(y, y_pred))
print(kmeans.score(X))

In [ ]:
kmeans = KMeans(n_clusters=5)
y_pred = kmeans.fit_predict(X)
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y_pred)

print(adjusted_rand_score(y, y_pred))
print(kmeans.score(X))

#### The shape matters

In [ ]:
X, y = make_blobs(random_state=170, n_samples=600)

random = np.random.RandomState(74)

transformation = random.normal(size=(2, 2))

X = np.dot(X, transformation)

ax = sns.scatterplot(x=X[:, 0], y=X[:, 1])

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_pred = kmeans.predict(X)

ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y_pred)
ax = sns.scatterplot(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], palette='orange', s=500)

In [ ]:
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
y_pred = kmeans.predict(X)

ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y_pred)
ax = sns.scatterplot(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], palette='orange', s=500)

### Agglomerative Clustering

How does it work?

1. Every point becomes a cluster.
2. The most "similar" clusters get combined into one.
3. Repeats step 2 until there are $k$ clusters.

In Sklean the definition of similarity is controlled by the `affinity` and `linkage` [hyperparameters](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html).

Disadvantages:
  - requires `k`
  - cannot deal with non-linear shapes
  - does not have a `.predict` method

![Alt](https://cdn-images-1.medium.com/max/800/1*ET8kCcPpr893vNZFs8j4xg.gif)

In [ ]:
X, y = make_blobs(random_state=1)

y_pred = AgglomerativeClustering(n_clusters=3).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

In [ ]:
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

y_pred = AgglomerativeClustering(n_clusters=2).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

### DBSCAN

While k-means and similar algorithms are **centroid-based**, Density-Based Spatial Clustering of Applications with Noise (DBSCAN) is a **density-based**
clustering algorithm. Instead of guessing how many clusters you need, by using DBSCAN,
you define two hyperparameters: $ϵ$ and $n$.

How does it work?

- two main parameters `eps` and `min_samples`;
- points get classified either as `core sample`, `boundary points`, or `noise`;
  - core samples are points with `min_samples` or more neighbors that are at distance `eps` or less;
  - boundary points are samples that are neighbors of a core sample in the cluster but are not themselves core samples (do not have `min_samples` neighbors);
  - noise is everything else.

Advantages:
  - does not require a number of clusters
  - can work with any shapes

Disadvantages:
  - slow
  - has two hyperparameters and choosing good
values for them (especially `eps`) could be challenging. Furthermore, having `eps` fixed, the clustering
algorithm cannot effectively deal with clusters of varying density.

More on DBSCAN: https://scikit-learn.org/stable/modules/clustering.html#dbscan, https://www.youtube.com/watch?v=RDZUdRSDOok

![Alt](https://dashee87.github.io/images/DBSCAN_tutorial.gif)

In [ ]:
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

y_pred = DBSCAN(eps=0.2).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

With an appropriate value for `eps`, DBSCAN can handle the strange three clusters we say earlier.

In [ ]:
X, y = make_blobs(random_state=170, n_samples=600)

random = np.random.RandomState(74)

transformation = random.normal(size=(2, 2))

X = np.dot(X, transformation)

dbscan = DBSCAN(eps=0.4)
clusters = dbscan.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=clusters, s=60)
plt.show()

### HDBSCAN 

The clustering algorithm that keeps the advantages of DBSCAN while removing the need to decide on the value of `eps`. The algorithm is capable of building clusters of
varying density.

It has one important hyperparameter: `n` - the minimum number of examples to put in a cluster. This hyperparameter is relatively simple to choose by intuition.

> **Note**: Because this algorithm is not implemented in sklearn, we have to install another package that provides it - https://pypi.org/project/hdbscan/.

Advantage:

- no hyperparameter tuning

In [ ]:
!pip install hdbscan

In [ ]:
from hdbscan import HDBSCAN

X, _ = make_blobs(1000)

y_pred = HDBSCAN(min_cluster_size=10).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

In [ ]:
X, y = make_blobs(random_state=170, n_samples=600)

random = np.random.RandomState(74)

transformation = random.normal(size=(2, 2))

X = np.dot(X, transformation)

y_pred = HDBSCAN(min_cluster_size=10).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

In [ ]:
X, y = make_moons(n_samples=200, noise=0.05, random_state=0)

y_pred = HDBSCAN(min_cluster_size=10).fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=60)
plt.show()

## Dimensionality Reduction

Algorithms for reducing the dimensionality of a dataset while preserving as much information as possible. The ultimate goal is to find the few `latent` (hidden) features that can be used to determine the outcome.

Two ways to do dimensionality reduction:
  - matrix factorization
    - **principle component analysis (PCA)**
    - linear autoencoder
    - latent dirichlet allocation
    - non-negative matrix factorization
    - generalized low rank models
    - Word2Vec
    - GloVe
  - neighbor graphs
    - laplacian eigenmaps
    - hessian eigenmaps
    - local tangent space alignment
    - JSE
    - isomap
    - **t-distributed stochastic neighbor embedding (t-SNE)**
    - locally linear embedding
    - **UMAP**